# Visualizations in R

Useful links
- [R Graph Gallery](https://www.r-graph-gallery.com)
- Libraries
    - [ggplot2](https://ggplot2.tidyverse.org)
    - [Plotly R](https://plotly.com/r/)


## Example data set
We'll be using a regulartory dataset that's made available to the public by the Home Mortgage Disclosure Act (HMDA). Congress enacted this legislation in 1975 in response to concerns about racial discrimination in mortgage financing. Each year thousands of mortgage financiers report almost every mortgage application they receive and the outcome of the loan (e.g., whether it was approved or rejected) along with some limited characteristics of the borrower(s) (e.g., race, ethnicity and gender) and some characteristics of the loan (e.g., interest rate, loan amount and length of the mortgage).

To learn more about the dataset, visit the Consumer Financial Protection Bureau (CFPB) [website](https://www.consumerfinance.gov/data-research/hmda/historic-data/), which hosts the public version for the HMDA data.

## Loading libraries
We'll load some libraries:
- **ggplot2**: Helpful library for making plots. Probably the most popular choice in R.
- **dplyr**: Helpful functions to manipulate data.
- **haven**: For loading `.dta` directly into R.
- **tidyr**: Manipulate data the tidyverse way.
- **labelled**: Manage labels from Stata datasets.

In [ ]:
library(ggplot2)
library(dplyr)    
library(haven)    
library(tidyr)    
library(labelled) 

## Loading the data
I've created a subset of the public dataset that has been compressed. The original dataset is 8.39 GB while this subset is just ~220 MB.

In [ ]:
# Define a variable to store this long URL that links to the data on Dropbox.
HMDA_URL <- "https://www.dropbox.com/s/8atgvmsq5ulssi4/hmda2019_extract.dta?dl=1"

# Load the entire dataset into a data.frame (technically, tbl_df)
hmda <- read_dta(HMDA_URL)

# Let's take a look at the first few rows
head(hmda)

## Scatter plot
Let's make a graph showing the disparity in interest rates between Black non-Latino and White Non-Latino borrowers by state.

In order to plot a scatter plot, we need two columns: one for the *x*-coordinate and one for the *y*-coordinate. Let's break down the steps we'll need to undertake:

- Restrict the sample to non-Latino Black and White borrowers.
- Calculate the mean interest rate spread for each group in each state.
- Reshape the data to have separate columns for each group.

In [ ]:
#----------------------------------------#
# Data Prep: Disparity in interest rates #
#----------------------------------------#

# Restrict sample, collapse data and then reshape
rate_spread_disparity_black_white <- hmda %>%
filter(race %in% c(4, 9) & ethnicity == 5) %>%
group_by(state_code, race) %>%
summarize(
    mean_spread = mean(rate_spread, na.rm = TRUE),
    count_spread = sum(sum(!is.na(rate_spread)))
) %>%
ungroup() %>%
pivot_wider(
    names_from = race,
    values_from = c(mean_spread, count_spread)
)

# Print the first few rows
head(rate_spread_disparity_black_white)

Now let's make our first scatter plot. It's a starting point that we'll refine in further steps.

In [ ]:
#-----------------------------------#
# Plot: Disparity in interest rates #
#-----------------------------------#

plt_disparity <- rate_spread_disparity_black_white %>%
  ggplot(aes(y = mean_spread_4, x = mean_spread_9)) +
  geom_point()

plt_disparity

Seems like we have a outlier for one of the states. Let's figure out what state it is.

In [ ]:
# Plot with labels

plt_disparity <- rate_spread_disparity_black_white %>%
  ggplot(aes(y = mean_spread_4, x = mean_spread_9)) +
  geom_text(aes(label = state_code), hjust = 0.5, vjust = 0.5)

plt_disparity

It's West Virgina! But what's going on? Let's take a look at the number of observations for West Virigina.

In [ ]:
# Print 15 smallest counts of Black borrowers

rate_spread_disparity_black_white %>%
  arrange(count_spread_4) %>%
  head(n = 15)

In [ ]:
# Print 15 smallest counts of White borrowers

rate_spread_disparity_black_white %>%
  arrange(count_spread_9) %>%
  head(n = 15)

There are some really small counts for some of the states. There are multiple ways to move forward. Maybe we should be using medians? Maybe there are some loan types with extreme values that we don't want to consider? For now, let's just restrict our plot to states with at least a 150 borrowers in each group.

In [ ]:
# Plot with labels states that have at least 150 borrowers

plt_disparity <- rate_spread_disparity_black_white %>%
  filter(count_spread_4 > 150 & count_spread_9 > 150) %>%
  ggplot(aes(y = mean_spread_4, x = mean_spread_9)) +
  geom_text(aes(label = state_code), hjust = 0.5, vjust = 0.5)

plt_disparity

We might want to emphasis data points with larger popluations. Let's allow the points to vary about the size of the total number of Black and White borrowers.

In [ ]:
# Plot states with points varying with count size

plt_disparity <- rate_spread_disparity_black_white %>%
  filter(count_spread_4 > 150 & count_spread_9 > 150) %>%
  mutate(count_spread = count_spread_4 + count_spread_9) %>%
  ggplot(aes(y = mean_spread_4, x = mean_spread_9)) +
  geom_point(aes(size = count_spread))

plt_disparity

Let's put some final touches on this graph.

- Filter the state "NA."
- Use transparency in the points to deal with overlapping points.
- Remove legend.
- Reduce ink-to-data ratio.
- Make the *y* and *x* axis the same scale.
- Add a diagonal line.
- Add titles, labels, footnote and annotations.

In [ ]:
# Add additional details for a more publication-ready plot

plt_disparity <- rate_spread_disparity_black_white %>%
  filter(count_spread_4 > 150 & count_spread_9 > 150) %>%
  filter(state_code != "NA") %>%
  mutate(count_spread = count_spread_4 + count_spread_9) %>%
  mutate(outlier = ifelse(mean_spread_4 > 3, state_code, "")) %>%
  ggplot(aes(y = mean_spread_4, x = mean_spread_9)) +
  geom_point(aes(size = count_spread), alpha = 0.5) +
  geom_text(aes(label = outlier), vjust = -1, hjust = 0.5) +
  geom_abline() +
  coord_fixed() +
  labs(
    title = "Rate spread for new mortgage borrowers, by state (2019)",
    caption = "Data Source: HMDA",
    x = "White borrowers",
    y = "Black borrowers"
  ) +
  xlim(0, 4) +
  ylim(0, 4) +
  annotate(
      geom = "text",
      x = 0.75,
      y = 3.9,
      label = "Black borrowers have higher spread",
  ) +
  annotate(
      geom = "text",
      x = 3.25,
      y = 0.1,
      label = "White borrowers have lower spread"
  ) +
  annotate(
      geom = "text",
      x = 3.4,
      y = 3.5,
      label = "Parity in spread between borrowers",
      angle = 45
  ) +
  theme_minimal() +
  theme(legend.position = "none")

plt_disparity


Still lots more that could be done (e.g., change font to match report/presentation and/or increase font size for readability). More importantly, you'd want to spend more time digging into the details and focusing on understanding the narrative you're trying to tell with the graph. But this is a good start!

## Bar chart

For this next exercise, we'll be plotting a bar graph. Let's investigate the disparity in loan approval rates between manufactured homes and site-built homes. In particular, the HMDA data provides information on the whether the covered loan or application is, or would have been in the case of applications, secured by a manufactured home and land, or by a manufactured home and not land. The variable is called in `mh_land_prop` and has the following values:

- 1 = Manufactured home and land
- 2 = Manufactured home and not land
- 1111 = Exempt

Let's take a quick look at this column by calculating a frequency table.

In [ ]:
# Tabulate frequency of loan type
table(hmda$mh_land_prop)

But what about site-built homes? Note that there is also another variable called `dwelling_category` with the following values:

- 1 = Manufactured
- 2 = Site-built

In [ ]:
# Tabulate frequency of loan type versus dwelling category (including NA)
table(hmda$mh_land_prop, hmda$dwelling_category, useNA = "always")

Now let's take a quick look at the approval variable, which is `approved`.

In [ ]:
# Tabulate frequency of approvals (including NA)
table(hmda$approved, useNA = "always")

So, we have a variable that's coded as zero and one. These values correspond to the following outcomes.

- 0 = Not approved
- 1 = Approved

It's helpful that the approval variable has already been coded as a binary outcome and has no missing values. We'll see why in the next step. Let's plot a bar chart!

In [ ]:
#----------------------------------------#
# Data Prep: Disparity in approval rates #
#----------------------------------------#

approval_disparity_mh_sb <- hmda %>%
  mutate(
    mh_land_prop_char = recode(
      mh_land_prop,
      `1` = "Manufactured home and land",
      `2` = "Manufactured home and not land",
      `1111` = "Exempt"
    )
  ) %>%
  replace_na(list(mh_land_prop_char = "Site-built")) %>%
  group_by(mh_land_prop_char) %>%
  summarise(frac_approved = mean(approved)) %>%
  ungroup()

approval_disparity_mh_sb

In [ ]:
#-----------------------------------#
# Plot: Disparity in approval rates #
#-----------------------------------#

approval_disparity_mh_sb %>%
ggplot(aes(x = mh_land_prop_char, y = frac_approved)) +
geom_col()

Let's do some clean-up!

- Make it a horizontal bar graph.
- Highlight the categories we want to emphasize.
- Reorder by the bar height.
- Use percent instead of fractions.
- Add labels.
- Reduce the ink-to-data ratio.

In [ ]:
# Add additional details for a publication-style graph

approval_disparity_mh_sb %>%
filter(mh_land_prop_char != "Exempt") %>%
mutate(
    bar_color = (mh_land_prop_char == "Site-built")
) %>%
ggplot(
    aes(
        x = frac_approved,
        y = reorder(mh_land_prop_char, -frac_approved),
        fill = bar_color
    )
) +
geom_col() +
scale_x_continuous(labels = scales::percent) +
labs(
    title = "Disparity in approval rates, by type of loan (2019)",
    caption = "Data Source: HMDA",
    x = "Percent of loans approved",
    y = ""
) +
scale_fill_manual(
    values = c("#1295D8", "darkgray"),
    guide = FALSE
) +
theme_minimal()

## Maps: Choropleth

Another common graph you'll see is a choropleth. There are advantages and disadvantages to using this type of graph. You may find it helpful to think of [alternative ways](https://www.vox.com/2016/6/2/11828628/election-maps-hard) of plotting your data instead of using a map. Nonetheless, choropleths are a popular way to visualize your data. There are many ways to visualize a choropleth in R. We'll start with the easy situation first, which is to plot data for a common geographic level (e.g., county). We want to plot the manufactured home loan applications as a percent of total mortgage applications by county.

In [ ]:
# Install and load package
if (!require("urbnmapr")) install.packages("urbnmapr")
library(urbnmapr)

Let's prep our data.

In [ ]:
#-----------------------------------------------------#
# Data Prep: Manfactured homes applications by county #
#-----------------------------------------------------#

# Calculate the percentage of mortgage applications that are manufactured homes
mh_by_ca_fips <- hmda %>%
  filter(state_code == "CA") %>%
  mutate(
    county_fips = substr(census_tract, 1, 5),
    mh = (dwelling_category == 1)
  ) %>%
  group_by(county_fips) %>%
  summarize(
      perc_mh = 100 * mean(mh),
      count_mh = sum(sum(!is.na(mh)))
)


Now let's prep our geographic data.

In [ ]:
# Add geographic data elements
mh_by_ca_fips_sf <- get_urbn_map("counties", sf = TRUE) %>%
  filter(state_name == "California") %>%
  left_join(
    mh_by_ca_fips,
    by = "county_fips"
  )


Finally, our map!

In [ ]:
#-----------------------------------------------#
# Plot: Manfactured home applications by county #
#-----------------------------------------------#

mh_by_ca_fips_sf %>%
  ggplot(aes()) +
  geom_sf(aes(fill = perc_mh)) +
  coord_sf(datum = NA) +
  guides(fill = guide_legend(title = "% of all applications")) +
  labs(
    title = "Manufactured home loan applications as a percent of total mortgage applications, by county",
    caption = "Data Source: HMDA (2019)"
  ) +
  theme(panel.background = element_blank())